In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



In [9]:
wine = pd.read_csv('./data/08_wine.csv')
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [20]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()



## 교차검증 : train_test_split() 방식

In [21]:
### 훈련데이터: 검증데이터 : 테스트 데이터 쪼개기
# - 보통 6 : 2 : 2 로 쪼갭니다
# - 보통 많이 사용하는 방식..

In [22]:
### 1. 훈련데이터와 테스트데이터를 8:2로 쪼개기

In [23]:
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size=0.2, random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [24]:
sub_input, val_input, sub_target, val_target = train_test_split(data,target,test_size=0.2, random_state=42)


print(sub_input.shape, sub_target.shape)
print(val_input.shape, val_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [25]:
from sklearn.preprocessing import StandardScaler

### 클래스(모델) 생성
ss = StandardScaler()

### 스스로 표준점수 생성
ss.fit(train_input)

### 훈련데이터를 표준점수로 변환하기
train_scaled = ss.transform(train_input)
### 테스트데이터를 표준점수로 변환하기
test_scaled = ss.transform(test_input)

In [28]:

from sklearn.tree import DecisionTreeClassifier

df = DecisionTreeClassifier(random_state=42)


df.fit(sub_input,sub_target)

print(df.score(sub_input,sub_target))

print(df.score(val_input,val_target))

0.996921300750433
0.8584615384615385


In [ ]:
##사전에 검증이 됬으면 테스트 진행함
# 테스트는 사용하지 않고 독립적으로 보관해놨다가 별개로 사용함
#검증테스트를 중간에 두고 사용함 
print(df.score(test_input, test_target))

## 교차검증 시작...(실제)

In [29]:
### 훈련데이터와 검증데이터를 구분할 필요없이
# - 내부적으로 구분해서 사용..
# - 내부적으로 구분에 사용하는 데이터는 훈련(train) 데이터
# - 데이터 준비는 기존처럼 훈련과 테스트 데이터만 준비
# - 검증데이터는 별도로 준비 안해도 됨..

In [ ]:
### 교차검증에 사용되는 주요 키워드
# 3-폴드(fold) 교차검증
#  :훈련데이터를 세부적으로 나눠서 데이터를 쪼개고 수행
# : 구간마다 훈련데이터를 검증데이터로 바꿔가면서 수행
# : k-폴드 또는 교차검증 또는 k-겹 교차검증이라고도 합니다.

# 보통 5-폴드 교차검증 또는 10 -폴드 교차검증을 사용합니다.
# : 훈련데이터로 80% 이상을 모두 모델 훈련에 사용 가능


In [ ]:
## 사용하는 데이터 다시한번 학인


In [31]:
### 교차검증 모듈 : cross_validate
from sklearn.model_selection import cross_validate

# df : 결정트리 훈련모델(다른 모델을 사용한 경우 해당 모델)
# 두번째 값: 훈련데이터(fold에서 훈련데이터 쪼갤때 사용)
# 세번째 값 : 검증데이터(fold에서 검증데이터 쪼갤때 사용)
scores = cross_validate(df,train_input, train_target)
#           1. 사용된 훈련모델,훈련데이터 만듦, 검증데이터만듦 

In [34]:
scores = cross_validate(df,train_input, train_target)
print(scores)
## 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print('최종 score = ' , np.mean(scores['test_score']))

{'fit_time': array([0.00698066, 0.00897598, 0.0079782 , 0.00598431, 0.00698256]), 'score_time': array([0.00099802, 0.00099683, 0.00099754, 0.00099707, 0.00099659]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
최종 score =  0.855300214703487


In [48]:
### 훈련데이터를 섞거나, 폴드의 갯수를 지정할 수 있는 클래스
# - StatifiedKFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수 그대로 사용
# cs : 분할기 속성
#    : 분할기로 StatifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold는 5, 기본 섞지는 않음

scores = cross_validate(df, train_input, train_target,
                                            cv = StratifiedKFold())


print(scores)
## 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print('최종 score = ' , np.mean(scores['test_score']))
##

{'fit_time': array([0.00697875, 0.00598478, 0.00598454, 0.00598383, 0.00499058]), 'score_time': array([0.00099802, 0.        , 0.00099611, 0.        , 0.00098443]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
최종 score =  0.855300214703487


In [52]:
### 훈련데이터를 섞거나, 폴드의 갯수를 지정할 수 있는 클래스
# - StatifiedKFold


# 교차검증 함수 그대로 사용
# cs : 분할기 속성
#    : 분할기로 StatifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold는 5, 기본 섞지는 않음

# n_splits : fold 갯수 , shuffle : 섞기
splitter = StratifiedKFold(n_splits=20, shuffle=True, random_state =42)
scores = cross_validate(df, train_input, train_target,cv = splitter)


print(scores)

## 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print('최종 score = ' , np.mean(scores['test_score']))


{'fit_time': array([0.0079813 , 0.00596952, 0.00801373, 0.00694704, 0.00698185,
       0.00696754, 0.00698566, 0.00699973, 0.0069623 , 0.00693655,
       0.00597739, 0.0069809 , 0.00598311, 0.0069809 , 0.00598335,
       0.00598502, 0.00598359, 0.00498629, 0.00598359, 0.00598335]), 'score_time': array([0.        , 0.00098729, 0.        , 0.        , 0.00101042,
       0.        , 0.00099468, 0.        , 0.        , 0.        ,
       0.0009973 , 0.00099778, 0.00099754, 0.00099802, 0.        ,
       0.00099659, 0.00099778, 0.00099778, 0.        , 0.        ]), 'test_score': array([0.85384615, 0.85384615, 0.88461538, 0.86923077, 0.84230769,
       0.86538462, 0.87307692, 0.85384615, 0.83461538, 0.84615385,
       0.87307692, 0.86923077, 0.85      , 0.86538462, 0.86153846,
       0.86538462, 0.84615385, 0.83783784, 0.8996139 , 0.86100386])}
최종 score =  0.8603073953073954


## 하이퍼파라미터 튜닝(AutoML)


In [53]:
###  교차검증, 하이퍼파라미터 찾기, 모델훈련을 한번에 자동으로 수행
# - 이런 개념을 AutoML 이라고 합니다.
#  - 사용패키지 : sklearn.model_selection
# - 사용클래스 : GridSearchCV
# - 그리드 서치라고도 합니다.

## 결정트리의 max_depth(트리의 깊이)값 찾기
##### - 그리드서치(GridSearchCV) 객체(모델) 사용


In [54]:
from sklearn.model_selection import GridSearchCV

#찾을 하이퍼파라미터는 딕셔너리로 
#실제 파라미터 변수이름 그대로, 찾을 범위 지정
params = {'max_depth' : range(5,20,1)}

#객체(모델 생성)
# - 첫번째 값 : 훈련모델 넣기
# - 두번째 값 : 찾을 하이퍼 파라미터 값들(딕셔너리로 정의)
# - 세번째 값 : CPU 코어 갯수( -1은 모든 코어 사용, 병렬 처리)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                  params, n_jobs = -1)

#훈련시키기
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [56]:
#그리드서치의 결과..
# - 하이퍼파라미터의 찾은 값 알려줌
# - 중요: 훈련모델이 생성되어 반환됨
#
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=8, random_state=42)


In [61]:
## 그리드서치를 통해 생성된 훈련모델을 통해
# 평가해보기, 훈련데이터 전체 사용
dt =gs.best_estimator_
print(dt.score(train_input, train_target))

0.9003271117952665


In [68]:
#가장 최적화된 하이퍼파라미터 값만 알고 싶으면
print(gs.best_params_)
#그리드 서치가 확인한 모든 값을 확인하고 싶으면
print('---------------------------------')
print(gs.cv_results_)
print('---------------------------------')
print(gs.cv_results_.keys())

{'max_depth': 8}
---------------------------------
{'mean_fit_time': array([0.00698156, 0.00678201, 0.00658197, 0.00718036, 0.00877624,
       0.01077132, 0.01077142, 0.00757995, 0.00817566, 0.00957375,
       0.00857735, 0.01116915, 0.00837698, 0.00997405, 0.00757942]), 'std_fit_time': array([0.00252274, 0.00074672, 0.00048906, 0.00097735, 0.00182775,
       0.00757978, 0.00509918, 0.00048933, 0.00097445, 0.00149291,
       0.00048842, 0.00506231, 0.00135317, 0.0018919 , 0.00048869]), 'mean_score_time': array([0.00119801, 0.00139651, 0.00099735, 0.00059853, 0.00079718,
       0.00119877, 0.00099587, 0.0007966 , 0.0007978 , 0.00059848,
       0.00059824, 0.00079894, 0.00099754, 0.00079703, 0.00059896]), 'std_score_time': array([4.01044101e-04, 4.89317611e-04, 4.10190833e-07, 4.88694526e-04,
       3.98589906e-04, 3.99710977e-04, 2.39844825e-06, 3.98309342e-04,
       7.45919956e-04, 7.98010831e-04, 4.88460995e-04, 3.99472787e-04,
       6.30901819e-04, 3.98517607e-04, 4.89045115e-04]),

In [69]:
### params 값 확인하기
print(gs.cv_results_['params'])

[{'max_depth': 5}, {'max_depth': 6}, {'max_depth': 7}, {'max_depth': 8}, {'max_depth': 9}, {'max_depth': 10}, {'max_depth': 11}, {'max_depth': 12}, {'max_depth': 13}, {'max_depth': 14}, {'max_depth': 15}, {'max_depth': 16}, {'max_depth': 17}, {'max_depth': 18}, {'max_depth': 19}]


In [70]:
### 1폴드 조회하기
print(gs.cv_results_['split0_test_score'])

[0.84711538 0.84807692 0.85769231 0.85288462 0.85769231 0.84519231
 0.85865385 0.86730769 0.86538462 0.86826923 0.86730769 0.86442308
 0.86346154 0.86634615 0.87211538]


In [72]:
### 훈련모델 최종검증하기
# 최종검증은 테스트 데이터 사용
print(dt.score(test_input,test_target))

0.8584615384615385


In [116]:
dfa=pd.DataFrame()
# dfa[]= gs.cv_results_['params']
# print(gs.cv_results_['split0_test_score'])
a= []
for i in gs.cv_results_['params']:
    for j in i.values():
        a.append(j)
        
dfa['key']=a
dfa['score']=gs.cv_results_['split0_test_score']
dfa.reset_index(drop=True)

print(dfa.loc[0])
dfa
        

        
    

key      5.000000
score    0.847115
Name: 0, dtype: float64


,key,score
0,5,0.847115
1,6,0.848077
2,7,0.857692
3,8,0.852885
4,9,0.857692
5,10,0.845192
6,11,0.858654
7,12,0.867308
8,13,0.865385
9,14,0.868269


## 찾을 하이퍼파라미터 여러개 사용하기

In [119]:
# min_impurity_decrease :결정트리 노드 분리시 불순도 감소 최소량
# max_dpth : 결정트리 깊이
# min_samples_split : 결정트리 노드를 나누기 위한 최소 샘플 갯수

params = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)}
##그리드 훈련시키기
gs = GridSearchCV(DecisionTreeClassifier(random_state =42),
                  params,n_jobs =-1)

gs.fit(train_input, train_target)


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [120]:
#밑 수치대로 작성하면 가장 최적화된 트리 값이 나온다.
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [121]:
print(dt.score(train_input,train_target))
print('-----------------------------')
print(dt.score(test_input,test_target))

0.9003271117952665
-----------------------------
0.8584615384615385


In [125]:
### 10-폴드 지정하기

splitter = StratifiedKFold(n_splits=7, shuffle=True, random_state =42)

params = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)}
##그리드 훈련시키기
gs = GridSearchCV(DecisionTreeClassifier(random_state =42),
                  params,cv=splitter, n_jobs =-1)

gs.fit(train_input, train_target)


GridSearchCV(cv=StratifiedKFold(n_splits=7, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [126]:
dt =gs.best_estimator_
print(dt.score(train_input,train_target))
print('-----------------------------')
print(dt.score(test_input,test_target))

0.9230325187608236
-----------------------------
0.8692307692307693


## 랜덤서치(Random Search)

In [127]:
### 랜덤서치 사용시 아래 조건 만족해야 합니다.
# - 하이퍼파라미터값이 수치인 것들만 사용
# - 범위나 간격을 미리 정하기 어려울 경우
# - 너무 많은 매개변수가 있어서 그리드서치 수행시간이 오래 걸리는 경우
# - 매개변수를 샘플링할때 확률분포객체를 전달..

# 사용모듈 : uniform(실수값) , randint(정수값)



In [137]:
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV


In [129]:
### 범위 설정
rgen = randint(0, 10)
rgen

In [131]:
###범위 내에서 임의값 출력하기(확률적으로 고르게 뽑아냅니다.)
print(rgen.rvs(100))

[8 9 7 5 5 8 8 0 1 8 7 5 2 6 9 5 0 8 9 9 2 2 0 3 9 0 5 9 9 3 2 5 1 3 5 1 2
 4 2 5 3 9 8 3 5 2 1 3 0 0 0 1 3 1 1 3 4 5 3 6 8 2 1 3 9 8 1 9 5 3 2 4 7 0
 3 3 0 2 3 2 6 0 0 1 1 8 5 3 4 1 5 6 2 8 5 5 7 3 6 0]


In [134]:
# 실수값
ugen = uniform(0,1)

#임의값 50개 조회

print(ugen.rvs(100))

[0.59849321 0.29215125 0.90179059 0.4216161  0.5156771  0.85781863
 0.15388062 0.64074493 0.79828061 0.67209661 0.26058881 0.50176997
 0.27690391 0.28022629 0.61422997 0.04794795 0.11620713 0.49798463
 0.34726287 0.13624111 0.79351232 0.82257937 0.56741617 0.36506161
 0.30276992 0.34013578 0.18120566 0.36465506 0.2192056  0.54990577
 0.81550333 0.93321235 0.5079296  0.31598731 0.39076398 0.66239378
 0.25390927 0.12311663 0.13627727 0.23760285 0.33424532 0.83486819
 0.78614454 0.61568673 0.05764247 0.11327641 0.57393923 0.84819986
 0.4610165  0.37727669 0.07434397 0.19506522 0.78651484 0.37718132
 0.31811868 0.57580647 0.83232801 0.43809576 0.86702775 0.3862063
 0.29794245 0.93133898 0.92300007 0.85298028 0.5443444  0.85857336
 0.73593128 0.95295061 0.72287969 0.58987445 0.41696778 0.07828592
 0.24759119 0.68562318 0.16816722 0.2540149  0.11873876 0.55481452
 0.22555215 0.8861293  0.87342494 0.90420256 0.43144594 0.70706694
 0.83541166 0.0874751  0.98568831 0.08146557 0.25355511 0.40153

In [143]:
### 하이퍼파라미터 찾을 매개변수 정의하기
params = {'min_impurity_decrease' : uniform(0.0001,0.001),
          'max_depth' :randint(20,50),
          'min_samples_split' : randint(2,25),}

# 랜덤서치 클래스 (모델) : RandomizedSearchCV()
# 사용방법은 그리드서치와 동일
# 결정트리 모델 사용, 훈련반복 횟수 100회, 코어 전체, 시드 42
# n_iter =n(훈련 반복횟수)
# 훈련 반복횟수를 적냐 안적냐는 range의 유무에 따라 틀림 ,range가 있으면 n_iter을 안적어도 됌

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state =42),
                  params,n_iter=100, n_jobs =-1,random_state=42)

gs.fit(train_input, train_target)


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000013134298700>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001312E27F040>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000131342A1A90>},
                   random_state=42)

In [144]:
print(gs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [148]:
print(gs.cv_results_['mean_test_score'])

[0.86203543 0.86203524 0.86453672 0.8639598  0.86434441 0.86184534
 0.86280669 0.86684608 0.86607592 0.86203561 0.85972662 0.8603054
 0.86338306 0.86145813 0.86222792 0.86145739 0.86761605 0.86261272
 0.86184238 0.86396109 0.86453709 0.86646165 0.86184312 0.86665322
 0.86165044 0.86030484 0.86299845 0.86742356 0.86203543 0.86838658
 0.86319186 0.86819334 0.85799659 0.86242023 0.86203561 0.86299752
 0.86261309 0.86049733 0.86857833 0.86299752 0.86800085 0.86203543
 0.8664635  0.86415451 0.86588473 0.86338362 0.8678091  0.86357611
 0.8624206  0.85857315 0.86203543 0.86742374 0.86607629 0.86838639
 0.86107333 0.86030466 0.86203543 0.86376805 0.86396146 0.86954098
 0.86954079 0.86800104 0.86145739 0.86242023 0.85972662 0.86261327
 0.86684626 0.86703913 0.86472958 0.86280503 0.86607814 0.86530632
 0.86800104 0.86203543 0.86088325 0.86261272 0.86299826 0.86242004
 0.86242023 0.86203543 0.86222773 0.86280558 0.8660787  0.85953413
 0.86800104 0.86145739 0.86184293 0.86588491 0.86184293 0.86761

In [149]:
### 훈련 100회 중에 가장 큰값이 최종 평가점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8695409787517583


In [150]:
dt =gs.best_estimator_
print(gs.best_params_)
print('-----------------------------')
print(dt.score(train_input,train_target))
print('-----------------------------')
print(dt.score(test_input,test_target))

{'max_depth': 29, 'min_impurity_decrease': 0.000437615171403628, 'min_samples_split': 16}
-----------------------------
0.8895516644217818
-----------------------------
0.8623076923076923


In [ ]:
#앙상블의 가장 기본적인 것은 결정트리 기반이다.